<a href="https://colab.research.google.com/github/Esantomi/theqoo-scraper/blob/main/theqoo_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 더쿠(Theqoo) 스크래핑 코드
- HOT 카테고리 게시글+댓글+메타데이터 스크래핑
- 작성자 : [Esantomi](https://github.com/Esantomi)

### 특정 페이지까지 게시글 정보 및 URL 수집

In [15]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import time

In [16]:
base_url = 'https://theqoo.net/index.php?mid=hot&filter_mode=normal&page='
num_pages = int(input("몇 페이지까지 크롤링할까요? "))

몇 페이지까지 크롤링할까요? 2


In [17]:
data = []
for page in range(1, num_pages+1):
    url = base_url + str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.select_one('#bd_801402415_0 > div > table')
    rows = table.select('tbody > tr')

    for row in rows:
        cols = row.select('td')
        row_data = {}
        for col in cols:
            col_class = col.get('class')
            if not col_class:
                continue
            col_class = col_class[0]

            col_text = col.text.strip()
            if col_class == 'title':
                link = col.select_one('a[href]')
                title_text = link.text.strip() if link else ''
                row_data['title'] = title_text

                if link:
                    href = link['href']
                    absolute_url = urljoin(base_url, href)
                    row_data['url'] = absolute_url
                    response = requests.get(absolute_url)
                    soup = BeautifulSoup(response.text, 'html.parser')

                    post_content = soup.select_one('.post_content')
                    post_content_text = post_content.text.strip() if post_content else ''
                    row_data['content'] = post_content_text

                    comments = soup.select('.comment_content')
                    comment_texts = [comment.text.strip() for comment in comments]
                    row_data['comment'] = '\n'.join(comment_texts)
            else:
                if col.select_one('a[href]'):
                    href = col.select_one('a[href]')['href']
                    absolute_url = urljoin(base_url, href)
                    response = requests.get(absolute_url)
                    soup = BeautifulSoup(response.text, 'html.parser')

                    post_content = soup.select_one('.post_content')
                    post_content_text = post_content.text.strip() if post_content else ''
                    row_data[col_class + '_content'] = post_content_text

                    comments = soup.select('.comment_content')
                    comment_texts = [comment.text.strip() for comment in comments]
                    row_data[col_class + '_comment'] = '\n'.join(comment_texts)
                else:
                    row_data[col_class] = col_text if col_text else None

        if row_data.get('no') not in ['전체공지', None]:
            data.append(row_data)

In [18]:
df = pd.DataFrame(data)
df = df.drop(['cate_content', 'cate_comment'], axis=1)
df.to_excel('theqoo_data.xlsx', index=False)
print(f"크롤링이 완료되었습니다. {len(df)}개의 게시물을 수집하였습니다.")

크롤링이 완료되었습니다. 58개의 게시물을 수집하였습니다.


### 개별 게시글 접속 후 크롤링 후 데이터프레임 병합
- 댓글이 빈 리스트로 수집되는 이슈가 있음.

In [19]:
# 엑셀 파일 불러오기
df = pd.read_excel("theqoo_data.xlsx")

In [20]:
import random

# 크롤링한 데이터를 저장할 빈 리스트 생성
results = []

# url 컬럼에서 주소 하나씩 가져오기
for url in df['url']:
    # requests를 사용하여 해당 주소의 html 코드 가져오기
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 게시글 내용 크롤링하기
    post_content = soup.select_one("div.rd.rd_nav_style2.clear > div.rd_body.clear > article").text.strip()

    # 댓글 내용 크롤링하기
    comments = soup.select("#cmtPosition > ul.fdb_lst_ul > li")
    comment_list = []
    for comment in comments:
        comment_id = comment.get('id')
        if comment_id and 'comment_' in comment_id:
            comment_text = comment.select_one('.xe_content').get_text(strip=True)
            comment_list.append(comment_text)

    # 결과 리스트에 해당 게시글 정보 추가하기
    results.append({'url': url, 'post_content': post_content, 'comments': comment_list})

    # 휴식
    time.sleep(random.uniform(0.1,1))

In [21]:
# 결과 리스트를 데이터프레임으로 변환하기
result_df = pd.DataFrame(results)

# 기존 데이터프레임과 병합하기
df['content'] = result_df['post_content']
df['comment'] = result_df['comments']

# 결과 데이터프레임을 엑셀 파일로 저장하기
df.to_excel("theqoo_data.xlsx", index=False)

### selenium 사용 시도

In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.4 MB/s eta 0:00:00


In [17]:
cat /etc/os-release

NAME="Ubuntu"
VERSION="20.04.5 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.5 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


In [13]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.nYllnjsiUy/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.w6oWupaP67/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Y4HknigiiX/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [18]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

url = "http://example.com/"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome("chromedriver", options=options)
driver.get(url)
print(driver.title)
driver.quit()

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://deb.debian.org/debian buster-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://deb.debian.org/debian-security buster/updates InRelease
Hit:9 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package

WebDriverException: ignored

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

url = "https://google.com" 
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

driver.get(url)
print(driver.title)
driver.quit()

<ipython-input-16-79211ffeea73>:9: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
<ipython-input-16-79211ffeea73>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


WebDriverException: ignored

In [7]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu0.20.04.3).
The following package was automatically installed and is no longer required:
  libnvidia-common-525
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [14]:
from selenium.webdriver import ChromeOptions
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

options = ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)


# 사이트 접속하기
driver.get('https://codeit.kr')

WebDriverException: ignored

In [1]:
# import packages
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import csv, time, sys

def create_driver():
    service = Service("./chromedriver")
    chrome_options = Options()
    chrome_options.page_load_strategy = 'eager'
    driver = Chrome(service=service,options = chrome_options)
    return driver

# Get Driver
driver = create_driver()

WebDriverException: ignored

In [ ]:
# 필요한 패키지 import
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

# 크롬 드라이버 경로 설정
chrome_driver_path = "/content/chromedriver.exe"
service = Service(chrome_driver_path)

# 엑셀 파일 불러오기
df = pd.read_excel("theqoo_data.xlsx")

# 크롤링한 데이터를 저장할 빈 리스트 생성
results = []

# Selenium으로 크롬 브라우저 열기
driver = webdriver.Chrome(service=service)

# url 컬럼에서 주소 하나씩 가져오기
for url in df['url']:
    # Selenium으로 해당 주소의 html 코드 가져오기
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    # 게시글 내용 크롤링하기
    post_content = driver.find_element_by_css_selector("div.rd.rd_nav_style2.clear > div.rd_body.clear > article").text.strip()

    # 댓글 내용 크롤링하기
    comments = driver.find_elements_by_css_selector("#cmtPosition > ul > li")
    comment_list = []
    for comment in comments:
        try:
            comment_text = comment.find_element_by_css_selector(".xe_content").text.strip()
        except:
            comment_text = comment.find_element_by_css_selector(".xe_wrap > .xe_content").text.strip()
        comment_list.append(comment_text)

    # 결과 리스트에 해당 게시글 정보 추가하기
    results.append({'url': url, 'post_content': post_content, 'comments': comment_list})

# 크롬 브라우저 닫기
driver.quit()

# 결과를 데이터프레임으로 변환하여 엑셀 파일에 저장
df_result = pd.DataFrame(results)
df_result.to_excel("theqoo_data.xlsx", index=False)

AttributeError: ignored